In [67]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
import tqdm
import os

In [5]:
driver = webdriver.Chrome(options=Options())

In [6]:
driver.get('https://genius.com/#top-songs')

In [9]:
num_clicks = 5 # you can change it to load more top songs!
driver.get('https://genius.com/#top-songs');
for n in range(5):
    button = driver.find_element(By.XPATH, '//*[@id="top-songs"]/div/div[3]/div')
    button.click()
    time.sleep(0.7)

page = driver.page_source

In [10]:
soup = BeautifulSoup(page)
boxes = soup.find_all('div', class_='PageGridFull-sc-18uuafq-0 kfrnFZ')[0]
box_links = boxes.find_all('a', href=True)
song_pages = []
for link in box_links:
    song_pages.append(link['href'])

In [40]:
songs_data = []
links = []
for link in song_pages:
    links.append(link)
    song_request = requests.get(link)
    song_soup = BeautifulSoup(song_request.content, 'lxml')

    song_name = song_soup.find('span', class_='SongHeaderdesktop__HiddenMask-sc-1effuo1-11 iMpFIj').text
    try:
        song_author = song_soup.find('a', class_='Link__StyledLink-rwn6i6-0 kMnVYG HeaderArtistAndTracklistdesktop__Artist-sc-4vdeb8-1 jhWHLb').text
    except:
        song_author = 'can not find author :('
    soup_song_info = song_soup.find_all('span', class_='LabelWithIcon__Label-hjli77-1 hgsvkF')
    if len(soup_song_info) < 3:
        continue
    song_date = soup_song_info[0].text
    song_num_viewers = soup_song_info[1].text
    try:
        song_num_views = soup_song_info[2].text
    except:
        song_num_views = 'no info :('

    song_dict = {
        'Name': song_name.replace('\u200b', '').replace('\n', ''),
        'Author': song_author,
        'Release date': song_date,
        '№ viewers': song_num_viewers,
        '№ views': song_num_views
    }

    songs_data.append(song_dict)

In [24]:
top_songs = pd.DataFrame(songs_data)

In [25]:
top_songs.to_csv('Genius_top_songs.csv')

In [17]:
songs_data

[{'Name': 'BornintheU.S.A.',
  'Author': 'Bruce Springsteen',
  'Release date': 'Jun. 4, 1984',
  '№ viewers': '27 viewers',
  '№ views': '1.7M views'},
 {'Name': 'vampire',
  'Author': 'Olivia Rodrigo',
  'Release date': 'Jun. 30, 2023',
  '№ viewers': '87 viewers',
  '№ views': '1.3M views'},
 {'Name': 'YOASOBI-アイドル(Idol)(Romanized)',
  'Author': 'Genius Romanizations',
  'Release date': 'Apr. 12, 2023',
  '№ viewers': '35 viewers',
  '№ views': '1.1M views'},
 {'Name': 'EndlessFashion',
  'Author': 'Lil Uzi Vert',
  'Release date': 'Jun. 30, 2023',
  '№ viewers': '23 viewers',
  '№ views': '269.7K views'},
 {'Name': 'hi.(ToxicGossipTrain)',
  'Author': 'Colleen Ballinger',
  'Release date': 'Jun. 28, 2023',
  '№ viewers': '47 viewers',
  '№ views': '101.3K views'},
 {'Name': 'Sprinter',
  'Author': 'Dave & Central Cee',
  'Release date': 'Jun. 1, 2023',
  '№ viewers': '43 viewers',
  '№ views': '580.2K views'},
 {'Name': 'Cupid(TwinVersion)',
  'Author': 'can not find author :(',
  

In [54]:
os.mkdir('song_images')

In [68]:
browser = webdriver.Chrome()
for link in tqdm.tqdm(links):
    img_name = link.split('/')[-1]
    browser.get(link)
    res = browser.page_source
    image_soup = BeautifulSoup(res, 'html.parser')
    img_elems = image_soup.find('img').get('src')
    img_response = requests.get(img_elems)
    with open(f"song_images/{img_name}.jpg", "wb") as f:
        f.write(img_response.content)

  8%|▊         | 5/60 [02:51<31:21, 34.20s/it]


KeyboardInterrupt: 